## Import and Preprocess the Dataset

In [21]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchsummary import summary

In [22]:
# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Data Preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)


Using device: cpu


In [23]:
len(train_dataset)

60000

In [24]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256

## Build Generator Model

In [25]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(100, 7*7*256),
            nn.BatchNorm1d(7*7*256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Unflatten(1, (256, 7, 7)),
            nn.ConvTranspose2d(256, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(64, 1, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.Tanh()
        )
    
    def forward(self, x):
        return self.main(x)

generator = Generator().to(device)

In [26]:
summary(generator, (100,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 12544]       1,266,944
       BatchNorm1d-2                [-1, 12544]          25,088
         LeakyReLU-3                [-1, 12544]               0
         Unflatten-4            [-1, 256, 7, 7]               0
   ConvTranspose2d-5            [-1, 128, 7, 7]         819,328
       BatchNorm2d-6            [-1, 128, 7, 7]             256
         LeakyReLU-7            [-1, 128, 7, 7]               0
   ConvTranspose2d-8           [-1, 64, 14, 14]         204,864
       BatchNorm2d-9           [-1, 64, 14, 14]             128
        LeakyReLU-10           [-1, 64, 14, 14]               0
  ConvTranspose2d-11            [-1, 1, 28, 28]           1,601
             Tanh-12            [-1, 1, 28, 28]               0
Total params: 2,318,209
Trainable params: 2,318,209
Non-trainable params: 0
---------------------------

## Build Discriminator Model

In [27]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=5, stride=2, padding=2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Flatten(),
            nn.Linear(128*7*7, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x)

discriminator = Discriminator().to(device)

In [28]:
summary(discriminator, (1, 28, 28))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 14, 14]           1,664
         LeakyReLU-2           [-1, 64, 14, 14]               0
           Dropout-3           [-1, 64, 14, 14]               0
            Conv2d-4            [-1, 128, 7, 7]         204,928
         LeakyReLU-5            [-1, 128, 7, 7]               0
           Dropout-6            [-1, 128, 7, 7]               0
           Flatten-7                 [-1, 6272]               0
            Linear-8                    [-1, 1]           6,273
           Sigmoid-9                    [-1, 1]               0
Total params: 212,865
Trainable params: 212,865
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.48
Params size (MB): 0.81
Estimated Total Size (MB): 1.29
-------------------------------------------

## Create Custom Loss Function

In [29]:
# Loss Function and Optimizers
criterion = nn.BCELoss()
optimizerG = optim.Adam(generator.parameters(), lr=1e-4)
optimizerD = optim.Adam(discriminator.parameters(), lr=1e-4)



## Create the Custom Training Loop

In [30]:
# Define the directory to save generated images
image_dir = './generated_images'
os.makedirs(image_dir, exist_ok=True)
# Training Loop
epochs = 300
noise_dim = 100
fixed_noise = torch.randn(16, noise_dim, device=device)

In [31]:
def train():
    for epoch in range(epochs):
        for images, _ in tqdm(train_loader):
            images = images.to(device)

            # Update Discriminator
            optimizerD.zero_grad()
            real_labels = torch.ones(images.size(0), 1, device=device)
            fake_labels = torch.zeros(images.size(0), 1, device=device)
            outputs = discriminator(images)
            d_loss_real = criterion(outputs, real_labels)
            d_loss_real.backward()

            noise = torch.randn(images.size(0), noise_dim, device=device)
            fake_images = generator(noise)
            outputs = discriminator(fake_images.detach())
            d_loss_fake = criterion(outputs, fake_labels)
            d_loss_fake.backward()
            optimizerD.step()

            # Update Generator
            optimizerG.zero_grad()
            outputs = discriminator(fake_images)
            g_loss = criterion(outputs, real_labels)
            g_loss.backward()
            optimizerG.step()

        print(f'Epoch [{epoch+1}/{epochs}] - Loss D: {d_loss_real + d_loss_fake:.4f}, Loss G: {g_loss:.4f}')

        # Save generated images after each epoch
        with torch.no_grad():
            generated_images = generator(fixed_noise).cpu()
            # Save the images to the specified directory
            vutils.save_image(generated_images, os.path.join(image_dir, f'epoch_{epoch+1}.png'), nrow=4, normalize=True)


In [ ]:
train()